<a href="https://colab.research.google.com/github/shonenkov/asr-shad/blob/feature-oof_metrics/Jasper_OOF_metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/shonenkov/asr-shad > /dev/null
!pip install -r './asr-shad/requirements.txt' > /dev/null

In [2]:
import sys
sys.path.insert(0, './asr-shad/utils')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [12]:
import pandas as pd
from metrics import mean_wer, mean_ser, levenshtein_distance

In [9]:
pred = pd.read_csv('/content/drive/My Drive/Sirius project. Numbers/waveforms/jasper_pred.csv')
pred_aug = pd.read_csv('/content/drive/My Drive/Sirius project. Numbers/waveforms/jasper_pred_aug.csv')

In [13]:
corpus = set(word for string in pred.reference for word in string.split())

def thousand(number: str) -> str:
    if number == 'одна':
        return 'тысяча'
    elif number in ['две', 'три', 'четыре']:
        return 'тысячи'
    else:
        return 'тысяч'

def correct(text: str) -> str:
    words = list(text.split())
    correct_words = []
    for i, word in enumerate(words):
        if word in corpus:
            correct_words.append(word)        
        else:
            correction = ''
            min_dist = len(word)
            for variant in corpus:
                if levenshtein_distance(word, variant) < min_dist:
                    min_dist = levenshtein_distance(word, variant)
                    correction = variant
            correct_words.append(correction)
                    
        if correct_words[-1][:5] == 'тысяч' and i > 0:
            correct_words[-1] = thousand(words[i - 1])      
        
    return ' '.join(correct_words)

In [ ]:
pred['edited'] = pred['prediction'].apply(correct)
pred_aug['edited'] = pred_aug['prediction'].apply(correct)

In [19]:
wer_base = mean_wer(pred.reference, pred.prediction)
ser_base = mean_ser(pred.reference, pred.prediction)

wer_base_correct = mean_wer(pred.reference, pred.edited)
ser_base_correct = mean_ser(pred.reference, pred.edited)

wer_aug = mean_wer(pred_aug.reference, pred_aug.prediction)
ser_aug = mean_ser(pred_aug.reference, pred_aug.prediction)

wer_aug_correct = mean_wer(pred_aug.reference, pred_aug.edited)
ser_aug_correct = mean_ser(pred_aug.reference, pred_aug.edited)

In [22]:
metric_df = pd.DataFrame({'base' : [wer_base, ser_base],
                          'with_correction' : [wer_base_correct, ser_base_correct],
                          'with_aug': [wer_aug, ser_aug],
                          'with_aug_and_corr': [wer_aug_correct, ser_aug_correct],
}, index=['wer', 'ser'])
metric_df.head()

,base,with_correction,with_aug,with_aug_and_corr
wer,0.013097,0.000841,0.000519,0.000519
ser,0.002226,0.000639,0.000430,0.000445


*То, что символьная метрика немножко подросла в конце, вполне может быть из-за различия метрик. Коррекция призвана уменьшить WER, поэтому, например, в ситуации, когда два слова склеились, мы исправим их на одно, при этом потеря слова дает вклад единичка в WER и большой вклад в SER*.